In [4]:
!pip install tmdbsimple

In [6]:
api_key = "4c8081ac8d03d88eb6332eabb7f3d950"
tmdb.API_KEY = api_key #This sets the API key setting for the tmdb object
search = tmdb.Search()

def get_movie_info_tmdb(movie):
    response = search.movie(query=movie)
    id=response['results'][0]['id']
    movie = tmdb.Movies(id)
    info=movie.info()
    return info

In [7]:
def get_movie_genres_tmdb(movie):
    response = search.movie(query=movie)
    id=response['results'][0]['id']
    movie = tmdb.Movies(id)
    genres=movie.info()['genres']
    return genres

def genlist(g):
  l=[]
  for i in g:
    l.append(i['name'])
  return l

In [8]:
def precision_recall(gt,preds):
    TP=0
    FP=0
    FN=0
    for t in gt:
        if t in preds:
            TP+=1
        else:
            FN+=1
    for p in preds:
        if p not in gt:
            FP+=1
    if TP+FP==0:
        precision=0
    else:
        precision=TP/float(TP+FP)
    if TP+FN==0:
        recall=0
    else:
        recall=TP/float(TP+FN)
    return precision,recall

In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline
import seaborn as sns
import pickle
import tmdbsimple as tmdb

df = pd.DataFrame()
df = pd.read_csv("/content/drive/My Drive/Multimodal Genre Classification/last.csv", engine="python")
 
from ast import literal_eval
df.genre_ids=df.genre_ids.apply(literal_eval)

df.genre_ids[0]

api_key = "d0ccc864ded48afb6a7e28b2d32001ed"
tmdb.API_KEY = api_key
#search = tmdb.Search()

genres=tmdb.Genres()
list_of_genres=genres.movie_list()['genres']
Genre_ID_to_name={}
for i in range(len(list_of_genres)):
    genre_id=list_of_genres[i]['id']
    genre_name=list_of_genres[i]['name']
    Genre_ID_to_name[genre_id]=genre_name

# genres=np.zeros((len(top1000_movies),3))
genres=[]
all_ids=[]
for i in range(len(df)):
    id=df.id[i]
    genre_ids=df.genre_ids[i]
    genres.append(genre_ids)
    all_ids.extend(genre_ids)

print(all_ids)

from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()
Y=mlb.fit_transform(genres)

print(Y.shape)
print(np.sum(Y, axis=0))

sample_overview=df.overview[5]
sample_title=df.title[5]
print("The overview for the movie",sample_title," is - \n\n")
print(sample_overview)

from sklearn.feature_extraction.text import CountVectorizer
import re

content=[]
for i in range(len(df)):
    id=df.id[i]
    overview=df.overview[i]
    overview=overview.replace(',','')
    overview=overview.replace('.','')
    content.append(overview)

vectorize=CountVectorizer(max_df=0.95, min_df=0.005)
X=vectorize.fit_transform(content)

print("SEE",X.shape)


import pickle
f4=open('X.pckl','wb')
f5=open('Y.pckl','wb')
pickle.dump(X,f4)
pickle.dump(Y,f5)
f6=open('Genredict.pckl','wb')
pickle.dump(Genre_ID_to_name,f6)
f4.close()
f5.close()
f6.close()

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)
print("After",X_tfidf.shape)


msk = np.random.rand(X_tfidf.shape[0]) < 0.8

X_train_tfidf=X_tfidf[msk]
X_test_tfidf=X_tfidf[~msk]
Y_train=Y[msk]
Y_test=Y[~msk]

positions=range(len(df))
# print positions
test_movies=np.asarray(positions)[~msk]
# test_movies

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report

parameters = {'kernel':['linear'], 'C':[0.01, 0.1, 1.0]}

genre_names = []
for id in df.genre_ids:
  for i in id:
    genre_name=Genre_ID_to_name[i]
    genre_names.append(genre_name)
genre_names = set(genre_names)

#New code for the next review

genre_list=sorted(list(Genre_ID_to_name.keys()))
print(genre_list)

from sklearn.naive_bayes import MultinomialNB
classifnb = OneVsRestClassifier(MultinomialNB())

classifnb.fit(X_train_tfidf, Y_train)
predsnb=classifnb.predict(X[~msk].toarray())

# classifnb.fit(X[msk].toarray(), Y_train)
# predsnb=classifnb.predict(X[~msk].toarray())

import pickle
f2=open('classifer_nb','wb')
pickle.dump(classifnb,f2)
f2.close()

print(classification_report(Y_test, predsnb, target_names=genre_names))

predictionsnb=[]
for i in range(X_test_tfidf.shape[0]):
    pred_genres=[]
    movie_label_scores=predsnb[i]
    for j in range(19):
        #print j
        if movie_label_scores[j]!=0:
            genre=Genre_ID_to_name[genre_list[j]]
            pred_genres.append(genre)
    predictionsnb.append(pred_genres)

for i in range(X_test_tfidf.shape[0]):
    if i%50==0 and i!=0:
        print('MOVIE: ',df.title[test_movies[i]],'\tPREDICTION: ',','.join(predictionsnb[i]))



[28, 35, 80, 28, 12, 35, 14, 10751, 28, 80, 53, 28, 18, 10752, 28, 878, 28, 18, 28, 18, 36, 10752, 28, 80, 18, 28, 35, 28, 53, 28, 12, 878, 28, 12, 878, 28, 28, 12, 35, 14, 28, 12, 35, 14, 878, 10751, 28, 878, 28, 35, 80, 28, 12, 878, 28, 12, 35, 28, 18, 53, 28, 12, 14, 28, 80, 18, 878, 53, 28, 12, 14, 28, 12, 16, 35, 10751, 28, 12, 14, 28, 12, 14, 28, 12, 878, 28, 12, 878, 28, 12, 878, 28, 53, 28, 53, 28, 12, 878, 28, 35, 27, 28, 80, 53, 28, 12, 878, 28, 12, 878, 28, 12, 14, 28, 80, 18, 53, 28, 53, 878, 28, 12, 35, 14, 28, 18, 36, 10752, 14, 12, 878, 28, 28, 12, 35, 14, 28, 18, 10752, 28, 12, 878, 53, 28, 12, 14, 28, 12, 35, 878, 28, 878, 28, 12, 35, 28, 12, 14, 878, 28, 12, 14, 878, 28, 12, 878, 28, 12, 53, 28, 12, 16, 35, 878, 28, 12, 14, 878, 28, 878, 28, 12, 35, 28, 12, 14, 878, 80, 28, 53, 28, 28, 80, 53, 28, 53, 878, 28, 878, 28, 12, 878, 53, 28, 12, 14, 10749, 10751, 28, 80, 18, 53, 28, 18, 878, 28, 12, 878, 28, 12, 14, 28, 9648, 878, 53, 28, 35, 53, 28, 12, 14, 28, 12, 14, 28,

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

          Music       0.31      0.36      0.33      1297
        Fantasy       0.33      0.24      0.28       916
          Crime       0.44      0.51      0.47      1804
            War       0.50      0.60      0.54      4512
       TV Movie       0.55      0.49      0.52      1947
          Drama       0.40      0.48      0.44      2115
      Animation       0.45      0.47      0.46      3481
        Romance       0.18      0.57      0.27       539
         Horror       0.52      0.61      0.56       744
        Mystery       0.39      0.47      0.42      2140
       Thriller       0.44      0.49      0.46      1613
         Comedy       0.51      0.83      0.64      2344
         Family       0.51      0.46      0.48      1261
         Action       0.29      0.33      0.31       920
        Western       0.58      0.66      0.62      1809
    Documentary       0.43      0.58      0.49      1908
Science Fiction       0.42    

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
type(X_tfidf)

scipy.sparse.csr.csr_matrix

In [ ]:
type(predsnb[0])

numpy.ndarray

In [ ]:
precs=[]
recs=[]
for i in range(len(test_movies)):
    if i%1==0:
        pos=test_movies[i]
        #test_movie=movies_with_overviews[pos]
        gtids=df.genre_ids[pos]
        gt=[]
        for g in gtids:
            g_name=Genre_ID_to_name[g]
            gt.append(g_name)
#         print predictions[i],movies_with_overviews[i]['title'],gt
        a,b=precision_recall(gt,predictionsnb[i])
        precs.append(a)
        recs.append(b)

print(np.mean(np.asarray(precs)),np.mean(np.asarray(recs)))
print("Average = ",(np.mean(np.asarray(precs))+np.mean(np.asarray(recs)))/2)

NameError: ignored

In [ ]:
print("f1-score", (2*(np.mean(np.asarray(precs)) * np.mean(np.asarray(recs)))) / (np.mean(np.asarray(precs)) + np.mean(np.asarray(recs))))

f1-score 0.48459070700859747


In [10]:
content2=[]
inp = input("ENTER THE MOVIE TO BE PREDICTED : ")

try:
  mov = df[df.title==inp].index.values.astype(int)[0]

except:
  print("Movie not found. Pulling information!")
  mov = 0
  x = get_movie_info_tmdb(inp)['overview']
  content2.append(x)

for i in range(len(df)):
    id=df.id[i]
    overview=df.overview[i]
    overview=overview.replace(',','')
    overview=overview.replace('.','')
    content2.append(overview)

vectorize=CountVectorizer(max_df=0.95, min_df=0.005)
Z=vectorize.fit_transform(content2)

predsnb=classifnb.predict(Z[mov].toarray())

predictionsnb=[]
pred_genres=[]
movie_label_scores=predsnb[0]
for j in range(19):
  if movie_label_scores[j]!=0:
    genre=Genre_ID_to_name[genre_list[j]]
    pred_genres.append(genre)
predictionsnb.append(pred_genres)


if(mov==0):
  y = get_movie_genres_tmdb(inp)
  actual = genlist(y)

else:
  g = df.genre_ids[df.title==inp]
  ind = g.keys()[0]
  actual = []
  for i in df.genre_ids[ind]:
    ga = Genre_ID_to_name[i]
    actual.append(ga)


print('MOVIE: ',inp,'\t -------> \tPREDICTION: ',','.join(predictionsnb[0]))
print('ACTUAL: ',','.join(actual))


ENTER THE MOVIE TO BE PREDICTED : Gone With The Wind
Movie not found. Pulling information!
MOVIE:  Gone With The Wind 	 -------> 	PREDICTION:  Drama,War
ACTUAL:  Drama,War,Romance
